In [1]:
import sys
import cv2
import numpy as np
import face_recognition
import os
import csv
import time

from datetime import datetime
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QPushButton, QLineEdit,
    QHBoxLayout, QStatusBar, QMenuBar, QAction, QTableWidget, QTableWidgetItem, QMessageBox,
    QStackedWidget, QGroupBox, QGraphicsView, QGraphicsScene, QSizePolicy, QGridLayout,
    QScrollArea
)
from PyQt5.QtGui import QImage, QPixmap, QIcon
from PyQt5.QtCore import QTimer, Qt, pyqtSignal
from stylesheet import get_dark_stylesheet, get_stylesheet


class ClickableUserPhoto(QLabel):
    clicked = pyqtSignal(str)

    def __init__(self, user_name, image_path):
        super().__init__()
        self.user_name = user_name
        self.setPixmap(QPixmap(image_path).scaled(100, 100, Qt.KeepAspectRatio, Qt.SmoothTransformation))
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("border: 2px solid gray; margin: 5px;")
        self.setFixedSize(110, 110)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.clicked.emit(self.user_name)

    def setSelected(self, selected):
        if selected:
            self.setStyleSheet("border: 2px solid green; margin: 5px;")
        else:
            self.setStyleSheet("border: 2px solid gray; margin: 5px;")

class FaceRecognitionApp(QMainWindow):
    def __init__(self):
        super().__init__()
             
        self.setWindowTitle("Face Recognition System")
        self.setGeometry(100, 100, 1200, 800)
        self.setWindowIcon(QIcon("icons/app_icon.png"))

        self.allowed_users = []
        self.user_photos = {}
        self.classNames = []

        self.init_face_recognition()
        self.init_ui()
        self.init_camera()

        self.apply_stylesheet()
        self.last_attendance_time = {}
        
    def init_ui(self):
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        
        main_layout = QHBoxLayout()
        self.central_widget.setLayout(main_layout)
    
        # Sidebar
        sidebar = QVBoxLayout()
        self.home_btn = QPushButton(QIcon("icons/home.png"), "Home")
        self.logs_btn = QPushButton(QIcon("icons/logs.png"), "Logs")
        self.settings_btn = QPushButton(QIcon("icons/settings.png"), "Settings")
        
        sidebar.addWidget(self.home_btn)
        sidebar.addWidget(self.logs_btn)
        sidebar.addWidget(self.settings_btn)
        sidebar.addStretch()
    
        sidebar_widget = QWidget()
        sidebar_widget.setLayout(sidebar)
        sidebar_widget.setFixedWidth(200)
    
        # Main content
        self.stacked_widget = QStackedWidget()
        
        # Home page
        home_page = QWidget()
        home_layout = QVBoxLayout()
        
        self.date_time_label = QLabel()
        self.date_time_label.setAlignment(Qt.AlignCenter)
        
        self.graphics_view = QGraphicsView()
        self.graphics_scene = QGraphicsScene()
        self.graphics_view.setScene(self.graphics_scene)
        self.graphics_view.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        
        input_group = QGroupBox("Capture")
        input_layout = QHBoxLayout()
        self.name_input = QLineEdit()
        self.name_input.setPlaceholderText("Enter your name")
        self.capture_button = QPushButton("Capture Photo")
        self.capture_button.setIcon(QIcon("icons/camera.png"))
        input_layout.addWidget(self.name_input)
        input_layout.addWidget(self.capture_button)
        input_group.setLayout(input_layout)
    
        # User selection area
        self.allowed_users_group = QGroupBox("Select Users")
        self.allowed_users_layout = QGridLayout()
        scroll_area = QScrollArea()
        scroll_widget = QWidget()
        scroll_widget.setLayout(self.allowed_users_layout)
        scroll_area.setWidget(scroll_widget)
        scroll_area.setWidgetResizable(True)
        allowed_users_main_layout = QVBoxLayout()
        allowed_users_main_layout.addWidget(scroll_area)
        self.allowed_users_group.setLayout(allowed_users_main_layout)
        self.allowed_users_group.setFixedHeight(200)  # Set fixed height to make it smaller
    
        home_layout.addWidget(self.date_time_label)
        home_layout.addWidget(self.graphics_view)
        home_layout.addWidget(input_group)
        home_layout.addWidget(self.allowed_users_group)
        
        home_page.setLayout(home_layout)
        
        # Logs page
        self.logs_page = QWidget()
        logs_layout = QVBoxLayout()
        self.table_widget = QTableWidget()
        logs_layout.addWidget(self.table_widget)
        self.logs_page.setLayout(logs_layout)
        
        # Add pages to stacked widget
        self.stacked_widget.addWidget(home_page)
        self.stacked_widget.addWidget(self.logs_page)
    
        main_layout.addWidget(sidebar_widget)
        main_layout.addWidget(self.stacked_widget)
    
        # Connect signals
        self.home_btn.clicked.connect(lambda: self.stacked_widget.setCurrentIndex(0))
        self.logs_btn.clicked.connect(self.show_logs)
        self.capture_button.clicked.connect(self.capture_photo)
    
        # Status bar
        self.status_bar = QStatusBar()
        self.setStatusBar(self.status_bar)
    
        # Menu bar
        self.menu_bar = QMenuBar()
        self.setMenuBar(self.menu_bar)
        
        file_menu = self.menu_bar.addMenu("File")
        help_menu = self.menu_bar.addMenu("Help")
        
        exit_action = QAction("Exit", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)
        
        about_action = QAction("About", self)
        about_action.triggered.connect(self.show_about_dialog)
        help_menu.addAction(about_action)
    
        # Date and time update
        self.date_time_timer = QTimer()
        self.date_time_timer.timeout.connect(self.update_date_time)
        self.date_time_timer.start(1000)
    
        self.update_allowed_users_list()
        
    def init_camera(self):
        self.cap = cv2.VideoCapture(0)
        # self.cap = cv2.VideoCapture('http://[2409:40d0:1008:f5c9:8000::]:8080/video')
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(20)

    def init_face_recognition(self):
        self.path = 'images'
        self.images = []
        self.classNames = []
        self.load_images()
        self.encodeListKnown = self.find_encodings(self.images)
        print('Encoding Complete')
    
    def load_images(self):
        myList = os.listdir(self.path)
        for cl in myList:
            curImg = cv2.imread(f'{self.path}/{cl}')
            self.images.append(curImg)
            self.classNames.append(os.path.splitext(cl)[0])

    def find_encodings(self, images):
        encodeList = []
        for img in images:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            encode = face_recognition.face_encodings(img)
            if encode:
                encodeList.append(encode[0])
        return encodeList

    def check_lighting(self, img, threshold=100):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        avg_brightness = np.mean(gray)
        return avg_brightness >= threshold, avg_brightness

    def update_allowed_users_list(self):
        for i in reversed(range(self.allowed_users_layout.count())): 
            widget = self.allowed_users_layout.itemAt(i).widget()
            if widget is not None: 
                widget.setParent(None)
        
        for i, className in enumerate(self.classNames):
            image_path = f'{self.path}/{className}.jpg'
            user_photo = ClickableUserPhoto(className, image_path)
            user_photo.clicked.connect(self.toggle_user_selection)
            user_photo.setSelected(className in self.allowed_users)
            self.user_photos[className] = user_photo
            self.allowed_users_layout.addWidget(user_photo, i // 4, i % 4)

    def toggle_user_selection(self, user_name):
        if user_name in self.allowed_users:
            self.allowed_users.remove(user_name)
            self.user_photos[user_name].setSelected(False)
        else:
            self.allowed_users.append(user_name)
            self.user_photos[user_name].setSelected(True)
        print(f"Allowed users: {self.allowed_users}")  # For debugging
    
    def update_frame(self):
        success, img = self.cap.read()
        if success:
            sufficient_light, avg_brightness = self.check_lighting(img)
            # Detect faces
            face_locations = face_recognition.face_locations(img)
            if face_locations:
                # Assume the first detected face
                top, right, bottom, left = face_locations[0]
                face_width = right - left
                distance = self.calculate_distance(face_width)

                if distance > 50:
                    img = self.apply_zoom(img, 2)  # Apply 2x zoom
                    
            if not sufficient_light:
                overlay = img.copy()
                height, width = overlay.shape[:2]
                cv2.rectangle(overlay, (0, height - 100), (width, height), (0, 0, 0), -1)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(overlay, "Low Light Detected", (20, height - 60), font, 1, (255, 255, 255), 2)
                cv2.putText(overlay, f"Brightness: {avg_brightness:.1f}", (20, height - 20), font, 0.7, (200, 200, 200), 1)
                alpha = 0.7
                img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

            imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

            facesCurFrame = face_recognition.face_locations(imgS)
            encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

            for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
                matches = face_recognition.compare_faces(self.encodeListKnown, encodeFace, tolerance=0.5)
                faceDis = face_recognition.face_distance(self.encodeListKnown, encodeFace)
                matchIndex = np.argmin(faceDis)

                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

                if matches[matchIndex]:
                    name = self.classNames[matchIndex]

                    if name.upper() in [user.upper() for user in self.allowed_users]:
                        if name not in self.verified_users:
            
                            if self.blink_counter >= self.blink_threshold:
                                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                                cv2.putText(img, f"{name} (Verified)", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                                self.mark_attendance(name)
                                self.verified_users.add(name)
                                self.blink_counter = 0
                                self.blink_start_time = None
                            elif elapsed_time > self.blink_timeout:
                                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 2)
                                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 255), cv2.FILLED)
                                cv2.putText(img, "Blink timeout, retry", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                                self.blink_counter = 0
                                self.blink_start_time = None
                            else:
                                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 2)
                                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 255), cv2.FILLED)
                                blinks_needed = self.blink_threshold - self.blink_counter
                                cv2.putText(img, f"Blink {blinks_needed} more", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                        else:
                            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                            cv2.putText(img, f"{name} (Verified)", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                            self.mark_attendance(name)
                    else:
                        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (255, 0, 0), cv2.FILLED)
                        cv2.putText(img, "Match found", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                else:
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
                    cv2.putText(img, "Not Verified", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            height, width, channel = img.shape
            step = channel * width
            qImg = QImage(img.data, width, height, step, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.graphics_scene.clear()
            self.graphics_scene.addPixmap(pixmap)


    def capture_photo(self):
        name = self.name_input.text().strip()
        if name:
            success, img = self.cap.read()
            if success:
                cv2.imwrite(f'{self.path}/{name}.jpg', img)
                self.images.append(img)
                self.classNames.append(name)
                self.encodeListKnown = self.find_encodings(self.images)
                self.update_allowed_users_list()
                self.status_bar.showMessage(f"Photo captured and saved for {name}", 5000)
            else:
                self.status_bar.showMessage("Failed to capture photo", 5000)
        else:
            self.status_bar.showMessage("Please enter a name before capturing", 5000)

    def mark_attendance(self, name):
        now = datetime.now()
        dtString = now.strftime('%H:%M:%S')
    
        # Check if 1 minutes have passed since last attendance
        if name in self.last_attendance_time:
            last_time = self.last_attendance_time[name]
            time_difference = now - last_time
            if time_difference.total_seconds() < 60:  # 60 seconds = 1 minutes
                return  

        # Update last attendance time
        self.last_attendance_time[name] = now

        with open('attendance.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([name, dtString])
        self.status_bar.showMessage(f"Attendance marked for {name}", 5000)

    def show_logs(self):
        self.stacked_widget.setCurrentIndex(1)
        self.load_attendance_data()

    def load_attendance_data(self):
        self.table_widget.clear()
        self.table_widget.setColumnCount(2)
        self.table_widget.setHorizontalHeaderLabels(["Name", "Time"])

        try:
            with open('attendance.csv', 'r') as f:
                reader = csv.reader(f)
                data = list(reader)
                self.table_widget.setRowCount(len(data))
                for i, row in enumerate(data):
                    for j, item in enumerate(row):
                        self.table_widget.setItem(i, j, QTableWidgetItem(item))
        except FileNotFoundError:
            self.status_bar.showMessage("No data found", 5000)

    def show_about_dialog(self):
        about_message = "Face Recognition  System\nVersion 1.0\nDeveloped by NitinRawat"
        QMessageBox.about(self, "About", about_message)

    def update_date_time(self):
        current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.date_time_label.setText(current_datetime)

    def apply_stylesheet(self):
        self.setStyleSheet(get_dark_stylesheet())

    def calculate_distance(self, face_width):
        # Assuming a known width of the face (e.g., 14cm) and focal length of the camera
        KNOWN_WIDTH = 14.0  # cm
        FOCAL_LENGTH = 600  # Adjust based on your camera

        distance = (KNOWN_WIDTH * FOCAL_LENGTH) / face_width
        return distance
    
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = FaceRecognitionApp()
    main_win.show()
    sys.exit(app.exec_())

Encoding Complete


AttributeError: 'FaceRecognitionApp' object has no attribute 'apply_zoom'

SystemExit: 0

C:\Users\jainr\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
